<a href="https://colab.research.google.com/github/visgraf/syntheticlearning/blob/main/notebooks/SynthDetStatistics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try:
  import datasetinsights
except ModuleNotFoundError:
  ! pip install datasetinsights

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Dataset Statistics for SynthDet
This example notebook shows how to use datasetinsights to load synthetic datasets generated from the [Perception package](https://github.com/Unity-Technologies/com.unity.perception) and visualize dataset statistics. It includes statistics and visualizations of the outputs built into the Perception package and should give a good idea of how to use datasetinsights to visualize custom annotations and metrics.

## Setup dataset
If the dataset was generated locally, replace `<GUID>` below with the name of the dataset folder, e.g. `f3763556-355f-4303-9acd-32334fda51aa`. Otherwise, see the instructions below for downloading datasets from Unity Simulation.

In [3]:
data_root = "/content/drive/MyDrive/datasets/d7a1a125-192b-49cc-ba04-d9dcf175efd4"

In [4]:
!ls $data_root

Datasetc3220d86-d46a-4ac0-a54b-0a5981a69940
Logs
RGB2622bb49-12c1-416a-8c78-ede7a767e763


### Unity Simulation [Optional]
If the dataset was generated on Unity Simulation, the following cells can be used to download the metrics needed for dataset statistics.

Provide the `run-execution-id` which generated the dataset and a valid `access_token` in the following cell. The `access_token` can be generated using the Unity Simulation [CLI](https://github.com/Unity-Technologies/Unity-Simulation-Docs/blob/master/doc/cli.md#usim-inspect-auth).

In [ ]:
from datasetinsights.io.downloader import UnitySimulationDownloader

#run execution id:
# run_execution_id = "xxx"
# #access_token:
# access_token = "xxx"
# #annotation definition id:
# annotation_definition_id = "6716c783-1c0e-44ae-b1b5-7f068454b66e"
# #unity project id
# project_id = "xxx"
# source_uri = f"usim://{project_id}/{run_execution_id}"

# downloader = UnitySimulationDownloader(access_token=access_token)

Before loading the dataset metadata for statistics we first download the relevant files from Unity Simulation.


In [ ]:
# downloader.download(source_uri=source_uri, output=data_root)

## Load dataset metadata
Once the dataset metadata is downloaded, it can be loaded for statistics using `datasetinsights.data.simulation`. Annotation and metric definitions are loaded into pandas dataframes using `AnnotationDefinitions` and `MetricDefinitions` respectively.

In [5]:
from datasetinsights.datasets.unity_perception import AnnotationDefinitions, MetricDefinitions


In [6]:
ann_def = AnnotationDefinitions(data_root)
ann_def.table

,id,name,description,format,spec
0,c31620e3-55ff-4af6-ae86-884aa0daa9b2,bounding box,Bounding box for each labeled object visible t...,json,"[{'label_id': 1, 'label_name': 'book_dorkdiari..."


In [7]:
metric_def = MetricDefinitions(data_root)
metric_def.table

,id,name,description,spec
0,db1b258e-d1d0-41b6-8751-16f601a2e230,scenario_iteration,Iteration information for dataset sequences,NaN
1,061e08cc-4428-4926-9933-a6732524b52b,Per Frame Foreground Object Placement Info,"Reports the world position, scale, rotation, a...",NaN
2,939248ee-668a-4e98-8e79-e7909f034a47,Per Frame Lighting Info,"Reports the enabled state, intensity, colour, ...",NaN
3,a4b2253c-0eb2-4a90-9b20-6e77f1d13286,Per Frame Camera Post Processing Info,Reports post-processing effects of cameras (or...,NaN
4,14adb394-46c0-47e8-a3f0-99e754483b76,random-seed,The random seed used to initialize the random ...,NaN
5,51da3c27-369d-4929-aea6-d01614635ce2,object count,Counts of objects for each label in the sensor...,"[{'label_id': 1, 'label_name': 'book_dorkdiari..."
6,659c6e36-f9f8-4dd6-9651-4a80e51eabc4,rendered object info,Information about each labeled object visible ...,"[{'label_id': 1, 'label_name': 'book_dorkdiari..."


## Built-in Statistics
The following tables and charts are supplied by `datasetinsights.data.datasets.statistics.RenderedObjectInfo` on datasets that include the "rendered object info" metric.

In [8]:
from datasetinsights.stats.statistics import RenderedObjectInfo
from datasetinsights.datasets.unity_perception.exceptions import DefinitionIDError
from datasetinsights.stats import bar_plot, histogram_plot, rotation_plot

max_samples = 10000          # maximum number of sample points used in histogram plots

rendered_object_info_definition_id = "659c6e36-f9f8-4dd6-9651-4a80e51eabc4"
roinfo = None
try:
    roinfo = RenderedObjectInfo(data_root=data_root, def_id=rendered_object_info_definition_id)
except DefinitionIDError:
    print("No RenderedObjectInfo in this dataset")

### Descriptive Statistics

In [9]:
print(roinfo.num_captures())
roinfo.raw_table.head(3)

1000


,label_id,instance_id,instance_color,visible_pixels,capture_id,annotation_id,sequence_id,step,label_name
0,35,83,"{'r': 0, 'g': 0, 'b': 19, 'a': 254}",1483,81f46e22-4411-4197-977a-90e4ad9186e9,None,51e8f3dc-a797-47b3-a0ec-5e3a3714f2f3,0,snack_breadsticks_nutella
1,48,19,"{'r': 255, 'g': 191, 'b': 0, 'a': 255}",1393,81f46e22-4411-4197-977a-90e4ad9186e9,None,51e8f3dc-a797-47b3-a0ec-5e3a3714f2f3,0,snack_salame_hillshire
2,14,4,"{'r': 255, 'g': 0, 'b': 223, 'a': 255}",5467,81f46e22-4411-4197-977a-90e4ad9186e9,None,51e8f3dc-a797-47b3-a0ec-5e3a3714f2f3,0,cleaning_freegentle_tide


### Total Object Count

In [10]:
total_count = roinfo.total_counts()
display(total_count)

display(bar_plot(
    total_count, 
    x="label_id", 
    y="count", 
    x_title="Label Name",
    y_title="Count",
    title="Total Object Count in Dataset",
    hover_name="label_name"
))

,label_id,label_name,count
0,1,book_dorkdiaries_aladdin,318
1,2,candy_minipralines_lindt,349
2,3,candy_raffaello_confetteria,328
3,4,cereal_capn_crunch,320
4,5,cereal_cheerios_honeynut,338
...,...,...,...
58,59,utensil_toiletpaper_scott,308
59,60,utensil_trashbag_valuecorner,326
60,61,vitamin_centrumsilver_adults,339
61,62,vitamin_centrumsilver_men,340


### Per Capture Object Count

In [11]:
per_capture_count = roinfo.per_capture_counts()
display(per_capture_count.head(10))

,capture_id,count
0,0025aaa4-fc2b-4ea4-8602-6ee3af810f33,31
1,006bb6ac-c571-48a2-8502-0ace1b8257ff,14
2,00a202a2-0d3e-4e06-9543-d112abe738e0,14
3,01261c1a-a1b0-415a-b49a-ac3c256dec46,33
4,01485042-3d0b-4ed1-b304-f3671ab173bc,24
5,015848ae-b590-4aac-83a4-999c932072e9,12
6,01bc3759-e2d3-40bc-b3dc-a509d5ced3f9,25
7,01cebd54-00c9-48e7-842d-8fc66e4c577d,20
8,01e93c91-bfe6-4692-aa36-f26c2a72be28,21
9,02021233-4bd0-44a6-8464-d6320f49d401,30


In [12]:
display(histogram_plot(
    per_capture_count, 
    x="count",  
    x_title="Object Counts Per Capture",
    y_title="Frequency",
    title="Distribution of Object Counts Per Capture",
    max_samples=max_samples
))

### Object Visible Pixels

In [13]:
display(histogram_plot(
    roinfo.raw_table, 
    x="visible_pixels",  
    x_title="Visible Pixels Per Object",
    y_title="Frequency",
    title="Distribution of Visible Pixels Per Object",
    max_samples=max_samples
))

## SynthDet Statistics
Metrics specific to the simulation can be loaded using `datasetinsights.datasets.unity_perception.Metrics`. 

### Foreground placement info

In [14]:
from datasetinsights.datasets.unity_perception import Metrics
import pandas as pd

metrics = Metrics(data_root=data_root)
foreground_placement_info_definition_id = "061e08cc-4428-4926-9933-a6732524b52b"

def read_foreground_placement_info(metrics=None, name=None, columns=None):
    filtered_metrics = metrics.filter_metrics(foreground_placement_info_definition_id)
    combined = pd.DataFrame(filtered_metrics[name].to_list(), columns=columns)
    
    return combined

In [15]:
rot_columns = ("x_rot", "y_rot", "z_rot")
orientation_info = read_foreground_placement_info(metrics=metrics, name="rotation", columns=rot_columns)
orientation_info.head(5)

,x_rot,y_rot,z_rot
0,287.3116,222.0825,346.5567
1,287.3116,222.0825,346.5567
2,287.3116,222.0825,346.5567
3,287.3116,222.0825,346.5567
4,287.3116,222.0825,346.5567


In [16]:
orientation_info.describe()

,x_rot,y_rot,z_rot
count,20217.000000,20217.000000,20217.000000
mean,175.349263,180.902260,175.571023
std,137.461299,101.239772,105.464936
min,0.533270,0.612660,0.000000
25%,46.633780,95.957650,81.916080
50%,85.569600,179.161900,176.180800
75%,314.262400,267.639400,268.626100
max,359.400800,359.908800,359.572000


In [17]:
rotation_plot(
    orientation_info,
    x="x_rot",
    y="y_rot",
    z="z_rot",
    title="Object orientations",
    max_samples=max_samples
)

In [18]:
histogram_plot(
    orientation_info, 
    x="x_rot",  
    x_title="Object Rotation (Degree)",
    y_title="Frequency",
    title="Distribution of Object Rotations along X direction",
    max_samples=max_samples
)

In [19]:
histogram_plot(
    orientation_info, 
    x="y_rot",  
    x_title="Object Rotation (Degree)",
    y_title="Frequency",
    title="Distribution of Object Rotations along Y direction",
    max_samples=max_samples
)

### Foreground Scale Info

In [20]:
scale_columns = ("x_scale", "y_scale", "z_scale")
scale_info = read_foreground_placement_info(metrics=metrics, name="scale", columns=scale_columns)
scale_info.head(5)

,x_scale,y_scale,z_scale
0,5.458627,5.458627,5.458627
1,7.080563,7.080563,7.080563
2,5.507010,5.507010,5.507010
3,4.793858,4.793858,4.793858
4,5.554353,5.554353,5.554353


In [21]:
histogram_plot(
    scale_info, 
    x="x_scale",  
    x_title="Scale",
    y_title="Frequency",
    title="Distribution of Foreground Scale along X direction",
    max_samples=max_samples
)

In [22]:
histogram_plot(
    scale_info, 
    x="y_scale",  
    x_title="Scale",
    y_title="Frequency",
    title="Distribution of Foreground Scale along Y direction",
    max_samples=max_samples
)

In [23]:
histogram_plot(
    scale_info, 
    x="z_scale",  
    x_title="Scale",
    y_title="Frequency",
    title="Distribution of Foreground Object Scale along Z direction",
    max_samples=max_samples
)

### Foreground Position Info

In [24]:
pos_columns = ("x_pos", "y_pos", "z_pos")
pos_info = read_foreground_placement_info(metrics=metrics, name="position", columns=pos_columns)
pos_info.head(5)

,x_pos,y_pos,z_pos
0,0.393073,0.367344,3
1,1.846960,0.497628,3
2,1.284966,1.072102,3
3,0.632993,1.592881,3
4,-0.143767,1.742321,3


In [25]:
histogram_plot(
    pos_info, 
    x="x_pos",
    x_title="Position",
    y_title="Frequency",
    title="Distribution of Foreground Position along X direction",
    max_samples=max_samples
)

In [26]:
histogram_plot(
    pos_info, 
    x="y_pos",
    x_title="Position",
    y_title="Frequency",
    title="Distribution of Foreground Position along Y direction",
    max_samples=max_samples
)

In [27]:
histogram_plot(
    pos_info, 
    x="z_pos",
    x_title="Position",
    y_title="Frequency",
    title="Distribution of Foreground Position along Z direction",
    max_samples=max_samples
)

### Lighting info 
There are 4 lights present in the SynthDet project. Statistics for each light are presented below.

In [28]:
lighting_info_definition_id = "939248ee-668a-4e98-8e79-e7909f034a47"
x_y_columns = ["x_rotation", "y_rotation"]
color_columns = ["color.r", "color.g", "color.b", "color.a"]
other_columns = ["intensity", "enabled", "lightName"]

def read_lighting_info(metrics, light_name=None):
    filtered_metrics = metrics.filter_metrics(lighting_info_definition_id)
    filtered_metrics = filtered_metrics.reset_index()
    colors = pd.json_normalize(filtered_metrics["color"])
    colors.columns = color_columns
    
    combined = pd.concat([filtered_metrics[x_y_columns], filtered_metrics[other_columns], colors], axis=1, join="inner")
    filter_combined = combined[combined["lightName"]==light_name]
    return filter_combined

#### Directional Light_Background_1_Intermittent

In [29]:
lighting_info = read_lighting_info(metrics, light_name="Directional Light_Background_1_Intermittent")
lighting_info.head(5)

,x_rotation,y_rotation,intensity,enabled,lightName,color.r,color.g,color.b,color.a
1,0.0,180.0,117.007843,True,Directional Light_Background_1_Intermittent,0.968222,0.832120,0.800875,1.0
5,0.0,180.0,74.515182,True,Directional Light_Background_1_Intermittent,0.735596,0.979877,0.803335,1.0
9,0.0,180.0,116.445496,False,Directional Light_Background_1_Intermittent,0.813855,0.732346,0.628997,1.0
13,0.0,180.0,74.050690,False,Directional Light_Background_1_Intermittent,0.701708,0.713679,0.565595,1.0
17,0.0,180.0,96.589607,False,Directional Light_Background_1_Intermittent,0.905098,0.667178,0.473243,1.0


In [30]:
rotation_plot(
    lighting_info,
    x="x_rotation",
    y="y_rotation",
    title="Light orientations",
    max_samples=max_samples
)

In [31]:
histogram_plot(
    lighting_info, 
    x="color.r",  
    x_title="Lighting Color",
    y_title="Frequency",
    title="Distribution of Lighting Color Redness",
    max_samples=max_samples
)

In [32]:
histogram_plot(
    lighting_info, 
    x="color.g",  
    x_title="Lighting Color",
    y_title="Frequency",
    title="Distribution of Lighting Color Greeness",
    max_samples=max_samples
)

In [33]:
histogram_plot(
    lighting_info, 
    x="color.b",  
    x_title="Lighting Color",
    y_title="Frequency",
    title="Distribution of Lighting Color Blueness",
    max_samples=max_samples
)

In [34]:
histogram_plot(
    lighting_info, 
    x="intensity",  
    x_title="Lighting Intensity",
    y_title="Frequency",
    title="Distribution of Lighting Intensity",
    max_samples=max_samples
)

In [35]:
histogram_plot(
    lighting_info, 
    x="enabled",  
    x_title="Lighting Enabled States (Bool)",
    y_title="Frequency",
    title="Distribution of Lighting Enabled States",
    max_samples=max_samples
)

#### Directional Light_All_0

In [36]:
lighting_info = read_lighting_info(metrics, light_name="Directional Light_All_0")
lighting_info.head(5)

,x_rotation,y_rotation,intensity,enabled,lightName,color.r,color.g,color.b,color.a
2,0.0,120.000008,0.560132,True,Directional Light_All_0,0.477598,0.900994,0.890212,1.0
6,0.0,120.000008,0.886445,True,Directional Light_All_0,0.698189,0.623554,0.798729,1.0
10,0.0,120.000008,0.659674,True,Directional Light_All_0,0.947412,0.484952,0.601518,1.0
14,0.0,120.000008,0.683432,True,Directional Light_All_0,0.684691,0.848101,0.956718,1.0
18,0.0,120.000008,0.759662,True,Directional Light_All_0,0.840501,0.896508,0.768621,1.0


In [37]:
rotation_plot(
    lighting_info,
    x="x_rotation",
    y="y_rotation",
    title="Light orientations",
    max_samples=max_samples
)

In [38]:
histogram_plot(
    lighting_info, 
    x="color.r",  
    x_title="Lighting Color",
    y_title="Frequency",
    title="Distribution of Lighting Color Redness",
    max_samples=max_samples
)

In [39]:
histogram_plot(
    lighting_info, 
    x="color.g",  
    x_title="Lighting Color",
    y_title="Frequency",
    title="Distribution of Lighting Color Greeness",
    max_samples=max_samples
)

In [40]:
histogram_plot(
    lighting_info, 
    x="color.b",  
    x_title="Lighting Color",
    y_title="Frequency",
    title="Distribution of Lighting Color Blueness",
    max_samples=max_samples
)

In [41]:
histogram_plot(
    lighting_info, 
    x="intensity",  
    x_title="Lighting Intensity",
    y_title="Frequency",
    title="Distribution of Lighting Intensity",
    max_samples=max_samples
)

In [42]:
histogram_plot(
    lighting_info, 
    x="enabled",  
    x_title="Lighting Enabled States (Bool)",
    y_title="Frequency",
    title="Distribution of Lighting Enabled States",
    max_samples=max_samples
)

#### Directional Light_All_1

In [43]:
lighting_info = read_lighting_info(metrics, light_name="Directional Light_All_1")
lighting_info.head(5)

,x_rotation,y_rotation,intensity,enabled,lightName,color.r,color.g,color.b,color.a
3,75.190041,326.852753,2.833493,True,Directional Light_All_1,0.559520,0.488970,0.596983,1.0
7,309.981476,259.730225,2.555454,True,Directional Light_All_1,0.423156,0.942220,0.997390,1.0
11,317.635162,197.917633,0.852361,True,Directional Light_All_1,0.595292,0.713915,0.558122,1.0
15,281.681183,231.168671,2.913534,True,Directional Light_All_1,0.530898,0.622522,0.560320,1.0
19,322.590942,212.160370,1.107904,True,Directional Light_All_1,0.715815,0.435193,0.488254,1.0


In [44]:
rotation_plot(
    lighting_info,
    x="x_rotation",
    y="y_rotation",
    title="Light orientations",
    max_samples=max_samples
)

In [45]:
histogram_plot(
    lighting_info, 
    x="color.r",  
    x_title="Lighting Color",
    y_title="Frequency",
    title="Distribution of Lighting Color Redness",
    max_samples=max_samples
)

In [46]:
histogram_plot(
    lighting_info, 
    x="color.g",  
    x_title="Lighting Color",
    y_title="Frequency",
    title="Distribution of Lighting Color Greeness",
    max_samples=max_samples
)

In [47]:
histogram_plot(
    lighting_info, 
    x="color.b",  
    x_title="Lighting Color",
    y_title="Frequency",
    title="Distribution of Lighting Color Blueness",
    max_samples=max_samples
)

In [48]:
histogram_plot(
    lighting_info, 
    x="intensity",  
    x_title="Lighting Intensity",
    y_title="Frequency",
    title="Distribution of Lighting Intensity",
    max_samples=max_samples
)

In [49]:
histogram_plot(
    lighting_info, 
    x="enabled",  
    x_title="Lighting Enabled States (Bool)",
    y_title="Frequency",
    title="Distribution of Lighting Enabled States",
    max_samples=max_samples
)

#### Directional Light_All_2

In [50]:
lighting_info = read_lighting_info(metrics, light_name="Directional Light_All_2")
lighting_info.head(5)

,x_rotation,y_rotation,intensity,enabled,lightName,color.r,color.g,color.b,color.a
0,0.0,180.0,0.400860,True,Directional Light_All_2,0.445944,0.720239,0.600485,1.0
4,0.0,180.0,0.303303,True,Directional Light_All_2,0.658419,0.405494,0.498517,1.0
8,0.0,180.0,0.368278,True,Directional Light_All_2,0.751961,0.811424,0.735853,1.0
12,0.0,180.0,0.452288,True,Directional Light_All_2,0.854151,0.873946,0.566815,1.0
16,0.0,180.0,0.428304,True,Directional Light_All_2,0.779275,0.420265,0.748195,1.0


In [51]:
rotation_plot(
    lighting_info,
    x="x_rotation",
    y="y_rotation",
    title="Light orientations",
    max_samples=max_samples
)

In [52]:
histogram_plot(
    lighting_info, 
    x="color.r",  
    x_title="Lighting Color",
    y_title="Frequency",
    title="Distribution of Lighting Color Redness",
    max_samples=max_samples
)

In [53]:
histogram_plot(
    lighting_info, 
    x="color.g",  
    x_title="Lighting Color",
    y_title="Frequency",
    title="Distribution of Lighting Color Greeness",
    max_samples=max_samples
)

In [54]:
histogram_plot(
    lighting_info, 
    x="color.b",  
    x_title="Lighting Color",
    y_title="Frequency",
    title="Distribution of Lighting Color Blueness",
    max_samples=max_samples
)

In [55]:
histogram_plot(
    lighting_info, 
    x="intensity",  
    x_title="Lighting Intensity",
    y_title="Frequency",
    title="Distribution of Lighting Intensity",
    max_samples=max_samples
)

In [56]:
histogram_plot(
    lighting_info, 
    x="enabled",  
    x_title="Lighting Enabled States (Bool)",
    y_title="Frequency",
    title="Distribution of Lighting Enabled States",
    max_samples=max_samples
)

### Camera Info

In [57]:
lighting_info_definition_id = "a4b2253c-0eb2-4a90-9b20-6e77f1d13286"
camear_columns = ["saturation", "contrast", "gaussianDofStart", "gaussianDofEnd", "gaussianDofRadius"]

def read_camera_info(metrics):
    filtered_metrics = metrics.filter_metrics(lighting_info_definition_id)

    return filtered_metrics[camear_columns]

In [58]:
camera_info = read_camera_info(metrics)
camera_info.head(5)

,saturation,contrast,gaussianDofStart,gaussianDofEnd,gaussianDofRadius
0,-5.628684,5.381949,3.273983,3.0,1.5
1,3.891687,9.698660,3.925745,3.0,1.5
2,-5.305204,-8.332521,2.623478,3.0,1.5
3,4.676216,8.962512,2.952379,3.0,1.5
4,-0.778224,9.324560,0.578761,3.0,1.5


In [59]:
histogram_plot(
    camera_info, 
    x="saturation",  
    x_title="Camera Saturation",
    y_title="Frequency",
    title="Distribution of Camera Saturation",
    max_samples=max_samples
)

In [60]:
histogram_plot(
    camera_info, 
    x="contrast",  
    x_title="Camera Contrast",
    y_title="Frequency",
    title="Distribution of Camera Contrast",
    max_samples=max_samples
)

In [61]:
histogram_plot(
    camera_info, 
    x="gaussianDofStart",  
    x_title="Camera Gaussian Depth of Field Start",
    y_title="Frequency",
    title="Distribution of Camera Gaussian Depth of Field Start",
    max_samples=max_samples
)

In [62]:
histogram_plot(
    camera_info, 
    x="gaussianDofEnd",  
    x_title="Camera Gaussian Depth of Field End",
    y_title="Frequency",
    title="Distribution of Camera Gaussian Depth of Field End",
    max_samples=max_samples
)

In [63]:
histogram_plot(
    camera_info, 
    x="gaussianDofRadius",  
    x_title="Camera Gaussian Depth of Field Radius",
    y_title="Frequency",
    title="Distribution of Camera Gaussian Depth of Field Radius",
    max_samples=max_samples
)

## Annotation Visualization
In the following sections we show how to load annotations from the Captures object and visualize them. Similar code can be used to consume annotations for model training or visualize and train on custom annotations.

### Unity Simulation [Optional]
If the dataset was generated on Unity Simulation, the following cells can be used to download the images, captures and annotations in the dataset. Make sure you have enough disk space to store all files. For example, a dataset with 100K captures requires roughly 300GiB storage.

In [ ]:
# downloader.download(source_uri=source_uri, output=data_root, include_binary=True)

### Load captures

In [64]:
from datasetinsights.datasets.unity_perception.captures import Captures
cap = Captures(data_root)
cap.captures.head(3)

,id,sequence_id,step,timestamp,sensor,ego,filename,format,annotations
0,d06b0378-3881-40ce-a3fa-fae0b5ec6855,8a33a538-8765-421f-9eea-1ca1304d06c7,0,0.0,{'sensor_id': '78f1e1ad-e927-49df-a7aa-95a2aa0...,{'ego_id': 'e2cf4315-e22a-48a2-8cea-14a6604526...,RGB2622bb49-12c1-416a-8c78-ede7a767e763/rgb_2.png,PNG,[{'id': '434fab7d-1028-4da8-a057-d07d59eb0ca4'...
1,47ba9886-6068-4e08-8cf3-af4313683980,717d2c0f-1466-4b82-9c42-e2d5eb9792ca,0,0.0,{'sensor_id': '78f1e1ad-e927-49df-a7aa-95a2aa0...,{'ego_id': 'e2cf4315-e22a-48a2-8cea-14a6604526...,RGB2622bb49-12c1-416a-8c78-ede7a767e763/rgb_3.png,PNG,[{'id': '5c6e92e8-ccc7-413d-b5aa-f41eb453dcb5'...
2,e1e0f645-a401-4e76-90f8-2952244e6b87,64d73f75-7c96-43fd-8d33-094fccfa1bff,0,0.0,{'sensor_id': '78f1e1ad-e927-49df-a7aa-95a2aa0...,{'ego_id': 'e2cf4315-e22a-48a2-8cea-14a6604526...,RGB2622bb49-12c1-416a-8c78-ede7a767e763/rgb_4.png,PNG,[{'id': '120ce271-5a4a-41f5-88fe-e5cedff53bc2'...


### Bounding Boxes
In this section we render 2d bounding boxes on top of the captured images.

In [65]:
import os

from ipywidgets import interact, interactive, fixed, interact_manual
from PIL import Image

from datasetinsights.stats.visualization.plots import plot_bboxes
from datasetinsights.datasets.synthetic import SynDetection2D,read_bounding_box_2d

bounding_box_definition_id = "c31620e3-55ff-4af6-ae86-884aa0daa9b2"
dataset = SynDetection2D(data_path=data_root, def_id=bounding_box_definition_id)
def draw_bounding_boxes(index):
    image, bboxes = dataset[index]
    return plot_bboxes(image, bboxes, dataset.label_mappings)

In [66]:
from ipywidgets import interact

# pick an index and visualize
interact(draw_bounding_boxes, index=list(range(len(dataset))))

interactive(children=(Dropdown(description='index', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14,…

<function __main__.draw_bounding_boxes>

### 3D Ground Truth Bounding Boxes
In this section we render 3d ground truth bounding boxes on top of the captured images.

In [67]:
import os
import numpy as np

from ipywidgets import interact
from PIL import Image
from datasetinsights.stats.visualization.plots import plot_bboxes3d
from datasetinsights.datasets.synthetic import read_bounding_box_3d

bounding_box_3d_defintion_id = "0bfbe00d-00fa-4555-88d1-471b58449f5c"
def draw_bounding_boxes3d(index):
    filename = os.path.join(data_root, box_captures.loc[index, "filename"])
    annotations = box_captures.loc[index, "annotation.values"]
    sensor = box_captures.loc[index, "sensor"]

    if 'camera_intrinsic' in sensor:
        projection = np.array(sensor["camera_intrinsic"])
    else:
        projection = np.array([[1,0,0],[0,1,0],[0,0,1]])

    image = Image.open(filename)
    boxes = read_bounding_box_3d(annotations)
    img_with_boxes = plot_bboxes3d(image, boxes, projection)
    img_with_boxes.thumbnail([1024,1024], Image.ANTIALIAS)
    display(img_with_boxes)

try:
    box_captures = cap.filter(def_id=bounding_box_3d_defintion_id)
    interact(draw_bounding_boxes3d, index=(0, box_captures.shape[0]))
except DefinitionIDError:
    print("No bounding boxes found")

No bounding boxes found


## Semantic Segmentation
In this section we render the semantic segmentation images on top of the captured images.

In [68]:
def draw_with_segmentation(index, opacity):
    filename = os.path.join(data_root, seg_captures.loc[index, "filename"])
    seg_filename = os.path.join(data_root, seg_captures.loc[index, "annotation.filename"])
    
    image = Image.open(filename)
    seg = Image.open(seg_filename)
    img_with_seg = Image.blend(image, seg, opacity)
    img_with_seg.thumbnail([1024,1024], Image.ANTIALIAS)
    display(img_with_seg)
    
try:
    semantic_segmentation_definition_id = "12f94d8d-5425-4deb-9b21-5e53ad957d66"
    seg_captures = cap.filter(def_id=semantic_segmentation_definition_id)
    interact(draw_with_segmentation, index=(0, seg_captures.shape[0]), opacity=(0.0, 1.0))
except DefinitionIDError:
    print("No semantic segmentation images found")

No semantic segmentation images found


## Instance Segmentation
In this section we render the instance segmentation images on top of the captured images. Image IDs are mapped to an RGBA color value, below the image we include a preview of the mapping between colors and IDs.

In [69]:
def instance_sorter(instance):
    return instance["instance_id"]

def draw_with_instance_segmentation(index, opacity):
    filename = os.path.join(data_root, inst_caps.loc[index, "filename"])
    seg_filename = os.path.join(data_root, inst_caps.loc[index, "annotation.filename"])

    image = Image.open(filename)
    seg = Image.open(seg_filename)
    img_with_seg = Image.blend(image, seg, opacity)
    img_with_seg.thumbnail([1024,1024], Image.ANTIALIAS)
    display(img_with_seg)

    anns = inst_caps.loc[index, "annotation.values"].copy()
    anns.sort(key=instance_sorter)

    count = min(5, len(anns))
    print("First {} ID entries:".format(count))

    for i in range(count):
        color = anns[i].get("color")
        print ("{} => Color({:>3}, {:>3}, {:>3})".format(anns[i].get("instance_id"), color.get("r"), color.get("g"), color.get("b")))

try:
    inst_seg_def_id = "1ccebeb4-5886-41ff-8fe0-f911fa8cbcdf"
    inst_caps = cap.filter(def_id=inst_seg_def_id)
    interact(draw_with_instance_segmentation, index=(0, inst_caps.shape[0]), opacity=(0.0, 1.0))
except DefinitionIDError:
    print("No instance segmentation images found")


No instance segmentation images found
